<a href="https://colab.research.google.com/github/ysooch0819/AI16-Projects/blob/main/N423a_Attention_ver_1_0_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 3 - assignmnet*

---





# N423. Seq2Seq with Attention Mechanism Assignment Plus

이번 시간에는 Korpora의 한국어 문답 데이터를 사용하여  
**어텐션 매커니즘이 적용된 Seq2Seq 모델**을 이용해 한국어 챗봇을 제작해보겠습니다.  

출처 : https://github.com/ko-nlp/Korpora  

#### **1) 데이터 로드 및 전처리**

In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
#현재 API를 사용하는 경우 pip 오류가 발생하므로 csv 형태의 Raw데이터를 그대로 가져와 사용하겠습니다.
corpus = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')

# 2,000개 데이터 사용 (Google Colab 일 경우 3,000개에서는 OOM 발생)
texts = []
pairs = []
for i, (text, pair) in enumerate(zip(corpus['Q'], corpus['A'])):
    texts.append(text)
    pairs.append(pair)
    if i >= 2000: 
        break 

# 데이터 체크
list(zip(texts, pairs))[1995:2000]

[('배 아프다', '약이 필요하면 도움을 받아보세요.'),
 ('배 터지겠네', '위를 좀 쉬게 해주세요.'),
 ('배 터지겠다.', '산책 좀 해야겠네여.'),
 ('배가 너무 고파', '뭐 좀 챙겨드세요.'),
 ('배가 넘넘 고파', '저도 밥 먹고 싶어요')]

In [3]:
#regular expression(regex)를 사용
import re

def cleaning_sent(sentence):
    '''
    한글 및 숫자, 물음표 및 공백을 제외하고 제거하는 함수입니다.

    Input:
        sentence : str. 정제하려는 문장을 입력으로 받습니다.
    Return:
        정제 완료된 문장 반환. str
    '''
    sent = re.sub(r'[^1-9가-힣? ]', '', sentence)
    return sent

Konlpy에서는 한국어 품사를 태그해주는 다양한 태거를 지원하고 있습니다.  
이 중에서 이번에는 트위터에서 제작한 오픈소스 한국어 처리기인 Okt를 활용하도록 하겠습니다.  

Konlpy 형태소 분석 및 품사 태깅 공식 문서 : https://konlpy.org/ko/v0.6.0/morph/

In [4]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 8.7 MB/s eta 0:00:00


In [5]:
from konlpy.tag import Okt
okt = Okt()

def morpheme_anlysis(sentence):
    '''
    문장이 들어왔을 때, 형태소 단위로 분석해주는 함수입니다.
    Input:
        sentence : str. 형태소 분석을 하려는 문장입니다.
    Return:
        형태소 단위로 뛰워쓰기 되어있는 문장 반환. str
    '''
    return ' '.join(okt.morphs(sentence))

In [6]:
def clean_and_morph(sentence, is_question=True):
    '''
    문장이 질문인지 대답인지에 맞추어 알맞은 형태로 변형해주는 함수입니다.

    Input:
        sentence : str. 변형하려는 문장입니다.
        is_question : 문장이 질문인지, 대답인지 알려주는 값입니다. 문장이면 True, 대답이면 False를 받습니다.

    Return:
        문장 타입에 따라 알맞게 변형된 문장을 반환합니다.
    '''

    sentence = cleaning_sent(sentence)
    sentence = morpheme_anlysis(sentence)

    if is_question:
        return sentence
    else:
        return ('<sos> ' + sentence, sentence + ' <eos>')

def preprocessing(questions, pairs):
    '''
    한국어 문답 데이터를 전처리하는 함수입니다.
    정제, 형태소 분석, 모델의 인풋,아웃풋으로 사용할 수 있는 형태로 변환하는 과정을 포함하고 있습니다.

    Input:
        questions : list. 문답 데이터 중 질문으로 이루어진 리스트를 받습니다.
        pairs : list. 문답 데이터 중 답변으로 이루어진 리스트를 받습니다.

    Return:
        질문, 인풋 형태의 문답, 아웃풋 형태의 문답
    '''
    answer_in = []
    answer_out = []
    
    pre_questions = [clean_and_morph(question, is_question=True) for question in questions]

    for pair in pairs:
        in_pair, out_pair = clean_and_morph(pair, is_question=False)
        answer_in.append(in_pair)
        answer_out.append(out_pair)

    return pre_questions, answer_in, answer_out

questions, answer_in, answer_out = preprocessing(texts, pairs)

#### **2) 토큰화**

전처리가 완료된 데이터를 모델에 넣을 수 있는 형태로 토큰화합니다.

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
#토크나이저 학습을 위해 전체 데이터 리스트를 하나 생성
all_sentence = questions + answer_in + answer_out

In [10]:
#리스트를 사용하여 tokenizer 학습
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentence)

In [11]:
#질문 답변 데이터 토큰화
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [12]:
#토큰화가 잘 적용되었는지 확인
print(questions[:3])
print(question_sequence[:3])

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다']
[[1758, 2493], [1609, 2494, 2495, 1610], [974, 1759, 1760, 213, 197, 106]]


In [13]:
#문장 최대 토큰 수 계산
#max_len은 모델이 생성할 수 있는 문장의 최대 길이도 되므로 고려해서 선정해야함
def len_cal(sentences):
    total_len = 0
    count = 0
    maxlen = 0

    for i in sentences:
        if maxlen < len(i):
            maxlen = len(i)
        total_len += len(i)
        count += 1

    print(total_len / count)
    print(maxlen)

len_cal(question_sequence)
len_cal(answer_in_sequence)
len_cal(answer_out_sequence)

4.228385807096452
12
5.888055972013993
20
5.888055972013993
20


In [14]:
#패딩
max_len = 20
question_pad = pad_sequences(question_sequence,
                             max_len,
                             padding= 'post')
answer_in_pad = pad_sequences(answer_in_sequence,
                             max_len,
                             padding= 'post')
answer_out_pad = pad_sequences(answer_out_sequence,
                             max_len,
                             padding= 'post')

In [15]:
#패딩 적용 확인
question_pad.shape, answer_in_pad.shape, answer_out_pad.shape

((2001, 20), (2001, 20), (2001, 20))

#### **3) 모델 구현**

**Keras Functional API**를 통해 모델을 구현해보겠습니다.  

1. 아래 설명을 잘 읽고 코드를 작성하여 모델을 완성해주세요!  
2. 모델이 잘 학습된다면 모델의 성능을 올리는 여러가지 기법을 적용하여 봅시다.  

    예시)
    - Dropout
    - Multiple LSTM Layer
    - ...

In [16]:
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense

In [17]:
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        '''
        Seq2Seq의 인코더입니다.
        
        Args:
            units (int) : 인코더 내부 lstm의 노드 수.
            vocab_size (int) : 임베딩 행렬의 단어 수. 없는 단어가 있으면 oov가 발생할 수 있습니다.
                -> 훈련하려는 문장의 단어는 모두 포함하고 있는 것이 좋다!
            embedding_dim (int) : 임베딩 차원 수. 복잡할수록 좋을 수도 있고, 아닐 수도 있습니다. 차원이 크면 보통 표현력이 좋다. but 용량이 커져서 안좋을 수 있다.
            time_steps (int) : 문장 토큰의 수. ex) 안녕하세요 조윤행입니다. -> 안녕하세요/ 조윤행/ 입니다/ -> 토큰 수 : 3개
        '''
        super().__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

In [18]:
class Attention(tf.keras.layers.Layer):
  def __init__(self):
      super().__init__()

  def call(self, query, values):
      #key와 value는 동일하므로 values 하나로 선언하고 사용합니다. transpose_b : v=values를 전치(transpose)해주기 위해 사용.
      scores = tf.matmul(query, values, transpose_b=True) # 쿼리 키 내적하여 상관관계 계산 (query vector) x (key vector)^ T
      distribution = tf.nn.softmax(scores/ scores.shape[-1]) # d_k로 나눠주고 softmax 취해서 소프트맥스 스코어 계산하기
      context_vector = tf.matmul(distribution, values) # 스코어 x value vector
      return context_vector

In [19]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        '''
        Seq2Seq의 디코더입니다.
        
        Args:
            units (int) : 디코더 내부 lstm의 노드 수.
            vocab_size (int) : 임베딩 행렬의 단어 수. 없는 단어가 있으면 oov가 발생할 수 있습니다.
                -> 훈련하려는 문장의 단어는 모두 포함하고 있는 것이 좋다!
            embedding_dim (int) : 임베딩 차원 수. 복잡할수록 좋을 수도 있고, 아닐 수도 있습니다. 차원이 크면 보통 표현력이 좋다. but 용량이 커져서 안좋을 수 있다.
            time_steps (int) : 문장 토큰의 수. 디코더에서는 최대로 생성할 수 있는 문장의 길이가 됩니다.
        '''
        super().__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True)
        #(attention) 어텐션 추가
        self.attention = Attention()
        self.dense = Dense(vocab_size, activation='softmax')

    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        
        # (attention) query_vector, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 query_vector를 생성합니다.        
        query_vector = tf.concat([initial_state[0][:, tf.newaxis, :], 
                               x[:, :-1, :]], axis=1)        
        # query_vector와 인코더에서 나온 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention(query_vector, encoder_inputs)
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)

        x = self.dense(x)
        return x, hidden_state, cell_state

In [20]:
class Seq2seq_with_Attention(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        """
        어텐션이 적용된 Seq2Seq 모델입니다. 인코더와 디코더를 선언합니다.
        """
        super().__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

    def call(self, inputs, training=True):
        """
        선언한 인코더와 디코더를 하나로 연결하여 Seq2Sqe with Attention 파이프라인을 구현합니다.
        
        Args:
            inputs : 문장의 단어 인덱스로 이루어진 데이터.
            training : True인 경우 교사강요를 사용하여 디코더의 입력값에 정답을 넣어주며, False인 경우 디코더의 출력 데이터를 입력으로 넣어주게 됩니다.
        """
        if training:
            encoder_inputs, decoder_inputs = inputs
            #(attention) 인코더가 context vector뿐만 아니라 모든 출력값을 만들도록 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            #(attention) 디코더가 인코더의 모든 출력값을 받도록 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs), context_vector)
            return decoder_outputs
        else:
            #(attention) 인코더가 context vector뿐만 아니라 모든 출력값을 만들도록 수정
            encoder_outputs, context_vector = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            for i in tf.range(self.time_steps):
                #디코더가 인코더의 모든 출력값을 받도록 수정
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq),
                                                                            context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis= -1),
                                         dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)

                if decoder_output == self.end_token:
                    break

                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

#### **4) 모델 훈련**

In [21]:
# 학습 파라미터
# 하이퍼 파라미터 튜닝 시 참고하기 쉽도록 한 셀에 모아 작성하는 것이 좋습니다.

BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = max_len
START_TOKEN = tokenizer.word_index['<sos>']
END_TOKEN = tokenizer.word_index['<eos>']

UNITS = 128

# padding을 포함하기위해 +1
VOCAB_SIZE = len(tokenizer.word_index) + 1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 5
NUM_EPOCHS = 20

In [22]:
# 모델 훈련을 위해 모델을 선언합니다.
seq2seq = Seq2seq_with_Attention(UNITS,
                  VOCAB_SIZE,
                  EMBEDDING_DIM,
                  TIME_STEPS,
                  START_TOKEN,
                  END_TOKEN)

seq2seq.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])

In [23]:
# 모델 훈련
seq2seq.fit([question_pad, answer_in_pad],
            answer_out_pad,
            epochs=100,
            batch_size=BATCH_SIZE,
            )

Epoch 1/100
126/126 [==============================] - 12s 10ms/step - loss: 2.7715 - acc: 0.7028
Epoch 2/100
126/126 [==============================] - 2s 12ms/step - loss: 1.7120 - acc: 0.7493
Epoch 3/100
126/126 [==============================] - 1s 10ms/step - loss: 1.6010 - acc: 0.7601
Epoch 4/100
126/126 [==============================] - 1s 10ms/step - loss: 1.5323 - acc: 0.7611
Epoch 5/100
126/126 [==============================] - 1s 10ms/step - loss: 1.4812 - acc: 0.7650
Epoch 6/100
126/126 [==============================] - 1s 10ms/step - loss: 1.4386 - acc: 0.7695
Epoch 7/100
126/126 [==============================] - 1s 10ms/step - loss: 1.3974 - acc: 0.7725
Epoch 8/100
126/126 [==============================] - 2s 13ms/step - loss: 1.3571 - acc: 0.7743
Epoch 9/100
126/126 [==============================] - 2s 13ms/step - loss: 1.3141 - acc: 0.7765
Epoch 10/100
126/126 [==============================] - 1s 10ms/step - loss: 1.2710 - acc: 0.7795
Epoch 11/100
126/126 [======

#### **5) 챗봇 구현**

In [24]:
# 인덱스를 단어로 변환하는 함수
def convert_index_to_text(indexs, end_token):

    sentence = ''
    
    for index in indexs:
        if index == end_token:
            break;
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
            sentence += ''
        
        sentence += ' '
    return sentence

In [25]:
# 문장을 입력받으면 대답을 만들어내는 함수를 제작합니다.
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

In [26]:
# 챗봇에서 입력한 문장을 모델에 넣을 수 있는 형태로 변환하는 함수
def question_to_input(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_pad = pad_sequences(question_sequence, maxlen= max_len, truncating='post', padding='post')
    return question_pad

# 간단한 챗봇 구현 -> q를 누르면 종료됩니다.
def run_chatbot(question):
    question_inputs = question_to_input(question)
    results = make_prediction(seq2seq, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

while True:
    user_input = input('대화를 입력하세요\n')
    if user_input =='q':
        break
    print('answer: {}'.format(run_chatbot(user_input)))

대화를 입력하세요
안녕?
answer: 저 도 반가워요 
대화를 입력하세요
안녕
answer: 저 도 즐거워요 
대화를 입력하세요
좋아
answer: 저 도 요 
대화를 입력하세요
ㅎㅎ
answer: 뭐라도 드세요 
대화를 입력하세요
밥먹었어?
answer: 저 는 배터리 가 밥 이 예요 
대화를 입력하세요
천잰데
answer: 제 가 더 천재 예요 
대화를 입력하세요
똑똑하구나
answer: 저 도 즐거워요 
대화를 입력하세요
즐겁ㄷ다
answer: 저 도 즐거워요 
대화를 입력하세요
좋아해
answer: 새로운 스타일 에 도전 하는 것 도 좋아요 
대화를 입력하세요
맞아
answer: 저 도 즐거워요 
대화를 입력하세요
q
